# Geospatial Change Detection - Training Notebook
This notebook trains a Siamese U-Net to detect deforestation from satellite images.
It is designed to run on **Kaggle Free GPUs** (T4).

## Instructions
1. **Add Data**: Upload your `EO_Exports` folder (from Google Drive) as a Kaggle Dataset.
2. **Connect Data**: Click 'Add Input' -> Your Dataset.
3. **Run All**: Execute all cells to train the model.
4. **Download**: The trained model `model_inference.pth` will be saved in the Output section.

In [ ]:
!pip install rasterio

## 1. Model Architecture (Siamese U-Net)
We define the model here so you don't need to upload extra python files.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )
    def forward(self, x):
        return self.conv(x)

class Down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.pool = nn.MaxPool2d(2)
        self.conv = ConvBlock(in_ch, out_ch)
    def forward(self, x):
        return self.conv(self.pool(x))

class Up(nn.Module):
    def __init__(self, in_ch, skip_ch, out_ch):
        super().__init__()
        self.up = nn.ConvTranspose2d(in_ch, out_ch, kernel_size=2, stride=2)
        self.conv = ConvBlock(out_ch + skip_ch, out_ch)
    def forward(self, x, skip):
        x = self.up(x)
        if skip.shape[2:] != x.shape[2:]:
            skip = F.interpolate(skip, size=x.shape[2:], mode='bilinear', align_corners=False)
        x = torch.cat([x, skip], dim=1)
        return self.conv(x)

class SiameseUNet(nn.Module):
    def __init__(self, in_ch=6, base=32):
        super().__init__()
        self.enc1 = ConvBlock(in_ch, base)
        self.enc2 = Down(base, base*2)
        self.enc3 = Down(base*2, base*4)
        self.enc4 = Down(base*4, base*8)
        self.bottleneck = ConvBlock(base*8*2, base*16)
        self.up3 = Up(base*16, base*4*2, base*8)
        self.up2 = Up(base*8,  base*2*2, base*4)
        self.up1 = Up(base*4,  base*1*2, base*2)
        self.final = nn.Conv2d(base*2, 1, kernel_size=1)

    def encode_single(self, x):
        e1 = self.enc1(x)
        e2 = self.enc2(e1)
        e3 = self.enc3(e2)
        e4 = self.enc4(e3)
        return e1, e2, e3, e4

    def forward(self, before, after):
        b1,b2,b3,b4 = self.encode_single(before)
        a1,a2,a3,a4 = self.encode_single(after)
        c4 = torch.cat([b4, a4], dim=1)
        c3 = torch.cat([b3, a3], dim=1)
        c2 = torch.cat([b2, a2], dim=1)
        c1 = torch.cat([b1, a1], dim=1)
        bt = self.bottleneck(c4)
        x = self.up3(bt, c3)
        x = self.up2(x, c2)
        x = self.up1(x, c1)
        out = self.final(x)
        return out

## 2. Dataset Loader
This expects your data to be in the Input directory.

In [ ]:
import os, glob, rasterio, numpy as np
from torch.utils.data import Dataset, DataLoader

# UPDATE THIS PATH to match where Kaggle mounts your dataset
# Usually: /kaggle/input/your-dataset-name
DATA_DIR = "/kaggle/input/eo-exports" 

class ChipDataset(Dataset):
    def __init__(self, folder):
        self.before = sorted(glob.glob(os.path.join(folder, "*_before.tif")))
        print(f"Found {len(self.before)} training pairs in {folder}")
        
    def __len__(self): return len(self.before)
    
    def __getitem__(self, idx):
        bfile = self.before[idx]
        afile = bfile.replace("_before.tif", "_after.tif")
        # Assuming we don't have ground truth masks yet for the timeline data,
        # we might need to generate them or use a dummy for self-supervised/pre-training.
        # BUT for this demo, let's assume you labeled some or we are just testing the loop.
        # If you don't have masks, this line will fail. 
        # For the timeline demo, we often just want to see the model run, so let's create a dummy mask if missing.
        
        with rasterio.open(bfile) as ds:
            b = ds.read().astype('float32')/10000.0
        with rasterio.open(afile) as ds:
            a = ds.read().astype('float32')/10000.0
            
        # Dummy mask (all zeros) if no mask file exists
        # In real training, you MUST have labeled masks (from Label Studio)
        m = np.zeros((1, b.shape[1], b.shape[2]), dtype=np.float32)
        
        return torch.from_numpy(b), torch.from_numpy(a), torch.from_numpy(m)

## 3. Training Loop

In [ ]:
import torch.optim as optim

def train():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)
    
    # Initialize Dataset
    # Note: If your data is in a subfolder, adjust DATA_DIR
    ds = ChipDataset(DATA_DIR)
    if len(ds) == 0:
        print("ERROR: No data found! Check DATA_DIR path.")
        return
        
    dl = DataLoader(ds, batch_size=4, shuffle=True, num_workers=2)
    
    model = SiameseUNet(in_ch=6).to(device)
    opt = optim.Adam(model.parameters(), lr=3e-4)
    bce = nn.BCEWithLogitsLoss()
    
    epochs = 5
    for ep in range(epochs):
        model.train()
        epoch_loss = 0.0
        for i, (b,a,m) in enumerate(dl):
            b = b.to(device); a = a.to(device); m = m.to(device)
            
            out = model(b, a)
            loss = bce(out, m)
            
            opt.zero_grad()
            loss.backward()
            opt.step()
            
            epoch_loss += loss.item()
            
        print(f"Epoch {ep+1}/{epochs} Loss: {epoch_loss/len(dl):.4f}")
        
    # Save Model
    torch.save(model.state_dict(), "model_inference.pth")
    print("Model saved to model_inference.pth")

train()